In [ ]:
# -*- coding: utf-8 -*-
# -*- author : Vincent Roduit -*-
# -*- date : 2023-11-25 -*-
# -*- Last revision: 2023-11-25 -*-
# -*- python version : 3.11.6 -*-
# -*- Description: Notebook that summarize results-*-

# <center> CS -433 Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Road Segmentation </center>
---

### Preparing environment for Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ml-project-2-team-slo/source

/content/drive/MyDrive/ml-project-2-team-slo/source


### Imports

In [ ]:
#import libraries
%matplotlib inline
import sklearn as sk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#import files
from load_datas import*
from constants import*
from visualization import*
from data_processing import*
from data_augmentation import*
from preprocessing_helper import *
from post_processing import*
from training_utils import*

# pytorch
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Load and preprocess datas

In [ ]:
imgs, gt_imgs = load_datas(NB_IMAGES_MAX)

imgs_train = imgs[:80]
gt_imgs_train = gt_imgs[:80]
imgs_test = imgs[80:100]
gt_imgs_test = gt_imgs[80:100]

In [ ]:
from new_pre_helper import*

In [ ]:
WINDOW = 60

In [22]:
X, y = new_image_generator(
    imgs_train,
    gt_imgs_train,
    window_size=WINDOW,
    num_samples=16000,
    batch_size=32)

x_val, y_val = new_create_windows_gt(
    imgs_test,
    gt_imgs_test,
    augm_patch_size=WINDOW)

Rotation for 45 degrees
Rotation for 135 degrees
Rotation for 225 degrees
Rotation for 315 degrees
Batch 1/500
Batch 51/500
Batch 101/500
Batch 151/500
Batch 201/500
Batch 251/500
Batch 301/500
Batch 351/500
Batch 401/500
Batch 451/500
end process...


In [24]:
train_dataloader = DataLoader(
    dataset=list(zip(X, y)),
    batch_size=32,
    shuffle=True,
    num_workers=0)

validate_dataloader = DataLoader(
    dataset=list(zip(x_val, y_val)),
    batch_size=32,
    shuffle=False,
    num_workers=0)

In [12]:
#Epoch 1/20, Loss: 0.6045116235923768, Validation Accuracy: 0.6547, F1 score: 0.5090

## After the first change in image_generator (only rotation)
#Epoch 1/20, Loss: 0.5709283715629577, Validation Accuracy: 0.7626, F1 score: 0.4797
## After second change in image_generator (rotation + flip)
#Epoch 1/20, Loss: 0.6519827550888061, Validation Accuracy: 0.6142, F1 score: 0.5411

In [25]:
# Set random seed for reproducibility
torch.manual_seed(0)

from cnn import *

cnn = Basic_CNN(WINDOW)

# Define the loss function, optimizer and scheduler
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(optimizer, scheduler, criterion, train_dataloader, validate_dataloader, num_epochs=20)

Epoch 1/20, Loss: 0.6433122103691101, Validation Accuracy: 0.6286, F1 score: 0.5560
Epoch 2/20, Loss: 0.5246402844810486, Validation Accuracy: 0.7473, F1 score: 0.6354
Epoch 3/20, Loss: 0.5011697268390656, Validation Accuracy: 0.7599, F1 score: 0.6432
Epoch 4/20, Loss: 0.4575054455947876, Validation Accuracy: 0.7964, F1 score: 0.6904
Epoch 5/20, Loss: 0.3875004348707199, Validation Accuracy: 0.8309, F1 score: 0.6917
Epoch 6/20, Loss: 0.3835271980243921, Validation Accuracy: 0.8486, F1 score: 0.7239
Epoch 7/20, Loss: 0.38002457772433756, Validation Accuracy: 0.8438, F1 score: 0.7083
Epoch 8/20, Loss: 0.40257974856261164, Validation Accuracy: 0.8459, F1 score: 0.7151
Epoch 9/20, Loss: 0.4339664003364352, Validation Accuracy: 0.8462, F1 score: 0.7321
Epoch 10/20, Loss: 0.4590229002073198, Validation Accuracy: 0.8211, F1 score: 0.6758
Epoch 00010: reducing learning rate of group 0 to 1.0000e-04.
Epoch 11/20, Loss: 0.5558509945775623, Validation Accuracy: 0.8445, F1 score: 0.7251
Epoch 12/2